In [83]:
import pandas as pd

In [84]:
from aicentro.session import Session
session = Session(verify=False)

In [85]:
import tensorflow as tf
import numpy as np
from tensorflow.python import keras
from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras.optimizers import RMSprop

In [86]:
batch_size = 50
num_classes = 3
epochs = 200

In [87]:
nvoc_df = pd.read_csv('/aihub/data/nvocdata_final_20200720.csv', encoding = 'UTF-8')

In [88]:
del nvoc_df['Unnamed: 0']

In [89]:
nvoc_df.corr()

,상담대분류,상담소분류,상품구분,영업단,급지,AS신청유형,AS원인유형,AS처리유형,cnslt_rscs_diff,as_rscs_diff,rscs_yn
상담대분류,1.000000,0.069312,-1.000000,0.028334,-0.093510,-0.026543,0.153577,0.027245,0.150649,0.129595,-0.200736
상담소분류,0.069312,1.000000,-0.069312,-0.055562,-0.057038,-0.047027,0.004922,-0.030815,-0.058146,-0.073440,0.021220
상품구분,-1.000000,-0.069312,1.000000,-0.028334,0.093510,0.026543,-0.153577,-0.027245,-0.150649,-0.129595,0.200736
영업단,0.028334,-0.055562,-0.028334,1.000000,-0.297507,-0.077719,0.092817,0.158349,-0.010757,0.026178,-0.020036
급지,-0.093510,-0.057038,0.093510,-0.297507,1.000000,0.022677,-0.050340,-0.014775,-0.068673,0.016906,-0.047739
AS신청유형,-0.026543,-0.047027,0.026543,-0.077719,0.022677,1.000000,0.003673,-0.006981,0.021067,-0.013123,0.009264
AS원인유형,0.153577,0.004922,-0.153577,0.092817,-0.050340,0.003673,1.000000,0.839792,0.025142,0.076443,-0.008181
AS처리유형,0.027245,-0.030815,-0.027245,0.158349,-0.014775,-0.006981,0.839792,1.000000,-0.009841,0.072479,-0.018618
cnslt_rscs_diff,0.150649,-0.058146,-0.150649,-0.010757,-0.068673,0.021067,0.025142,-0.009841,1.000000,0.504470,-0.619433
as_rscs_diff,0.129595,-0.073440,-0.129595,0.026178,0.016906,-0.013123,0.076443,0.072479,0.504470,1.000000,-0.341389


In [90]:
nvoc_df

,상담대분류,상담소분류,상담일자,상품구분,영업단,급지,해지일,AS신청유형,AS원인유형,AS처리유형,AS완료일자,AS처리시간,cnslt_rscs_diff,as_rscs_diff,rscs_yn
0,1,9,2016-01-08,0,1,4,2016-10-14,15,62,57,2016-01-08,28,280,280.0,False
1,1,9,2016-01-09,0,0,2,2017-04-17,25,62,57,2016-01-09,24,464,464.0,False
2,1,9,2016-01-09,0,0,2,2017-04-17,25,86,40,2014-02-18,24,464,1154.0,False
3,1,9,2016-01-09,0,0,2,2017-04-17,25,0,1,2016-01-08,1,464,465.0,False
4,1,9,2016-01-09,0,0,2,2017-04-17,13,62,57,2015-11-30,25,464,504.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2117,1,0,2020-06-10,0,1,2,2020-06-15,25,81,76,2018-08-31,2,5,654.0,True
2118,1,0,2020-06-10,0,1,2,2020-06-15,23,49,0,2016-07-29,39,5,1417.0,True
2119,1,0,2020-06-10,0,1,2,2020-06-15,18,62,57,2017-11-01,0,5,957.0,True
2120,1,0,2020-06-10,0,1,2,2020-06-15,15,62,57,2015-11-12,0,5,1677.0,True


In [123]:
val_columns = ['상담대분류', '상담소분류', 'AS신청유형', 'AS원인유형', 'AS처리유형', 'rscs_yn']
nvoc_df = pd.DataFrame(nvoc_df, columns=val_columns)

In [124]:
nvoc_df

,상담대분류,상담소분류,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,1,9,15,62,57,False
1,1,9,25,62,57,False
2,1,9,25,86,40,False
3,1,9,25,0,1,False
4,1,9,13,62,57,False
...,...,...,...,...,...,...
2117,1,0,25,81,76,True
2118,1,0,23,49,0,True
2119,1,0,18,62,57,True
2120,1,0,15,62,57,True


In [125]:
## 데이터를 읽고 순서를 섞는다
nvoc_model_df=nvoc_df.sample(frac=1).reset_index(drop=True)
nvoc_model_df

,상담대분류,상담소분류,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,1,0,23,72,61,False
1,1,5,23,57,48,False
2,1,2,18,57,48,False
3,1,9,23,70,61,False
4,1,5,18,62,50,False
...,...,...,...,...,...,...
2117,1,11,11,70,61,False
2118,1,2,25,3,3,False
2119,1,2,18,57,48,False
2120,1,2,23,62,57,False


In [126]:
nvoc_model_df.dtypes

상담대분류      int64
상담소분류      int64
AS신청유형     int64
AS원인유형     int64
AS처리유형     int64
rscs_yn     bool
dtype: object

In [127]:
nvoc_train=nvoc_model_df.iloc[0:1000,:]
nvoc_test=nvoc_model_df.iloc[1000:2121,:]

In [105]:
x_train= nvoc_train.iloc[:,0:7].values
x_test= nvoc_test.iloc[:,0:7].values
y_train= nvoc_train.iloc[:,7:8]
y_test= nvoc_test.iloc[:,7:8]

In [110]:
y_train

,rscs_yn
0,False
1,False
2,True
3,False
4,False
...,...
995,False
996,False
997,True
998,False


In [111]:
# encoder={k:v for v,k in enumerate(y_train.drop_duplicates())}
# encoder
sets=nvoc_model_df.iloc[:,7:8].drop_duplicates()["rscs_yn"].tolist()
encoder={k:v for v,k in enumerate(sets)}
y_train=[ encoder[i] for i in y_train["rscs_yn"].tolist() ]
y_train = keras.utils.to_categorical(y_train, num_classes)

y_test=[ encoder[i] for i in y_test["rscs_yn"].tolist() ]
y_test = keras.utils.to_categorical(y_test, num_classes)

In [112]:
y_test

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [113]:
# 데이터 shape 을 확인해 봅니다.
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(1000, 7) (1000, 3) (1121, 7) (1121, 3)


In [118]:
model = Sequential()
model.add(Dense(4, activation='relu', input_shape=(7,)))
#model.add(Dropout(0.2))
model.add(Dense(5, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(5, activation='relu'))
#model.add(Dropout(0.2))

model.add(Dense(num_classes, activation='softmax'))

model.summary()
#'binary_crossentropy'
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 4)                 32        
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 25        
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 18        
Total params: 105
Trainable params: 105
Non-trainable params: 0
_________________________________________________________________


In [119]:
import  tensorflow.keras.optimizers
dir( tensorflow.keras.optimizers)

['Adadelta',
 'Adagrad',
 'Adam',
 'Adamax',
 'Ftrl',
 'Nadam',
 'Optimizer',
 'RMSprop',
 'SGD',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_deprecation_wrapper',
 '_sys',
 'deserialize',
 'get',
 'schedules',
 'serialize']

In [120]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 1000 samples, validate on 1121 samples


W0721 11:28:45.220594 139801898121024 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/200
  50/1000 [>.............................] - ETA: 3s - loss: 0.9877 - acc: 0.5933

ValueError: could not convert string to float: '40..99'

In [121]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.2500826951438673
Test accuracy: 0.5732977
